# Provider Packages 📦

In [1]:
from dotenv import load_dotenv
import os

load_dotenv()

True

## Anthropic
### General hosting via Anthropic
Note that it will **by default take the enviroment variable ANTHROPIC_API_KEY** as API key

- Python SDK Docs: https://github.com/anthropics/anthropic-sdk-python

#### NORMAL CALL

In [2]:
import anthropic

client = anthropic.Anthropic()
message = client.messages.create(
    model="claude-sonnet-4-20250514",
    max_tokens=1024,
    messages=[
        {"role": "user", "content": "Hello, Claude"}
    ]
)
message.model_dump()

{'id': 'msg_01KShvuKMU8of3Z93HXA3NYf',
 'content': [{'citations': None,
   'text': "Hello! It's nice to meet you. How are you doing today? Is there anything I can help you with or would you like to chat about?",
   'type': 'text'}],
 'model': 'claude-sonnet-4-20250514',
 'role': 'assistant',
 'stop_reason': 'end_turn',
 'stop_sequence': None,
 'type': 'message',
 'usage': {'cache_creation_input_tokens': 0,
  'cache_read_input_tokens': 0,
  'input_tokens': 10,
  'output_tokens': 34,
  'server_tool_use': None,
  'service_tier': 'standard'}}

#### TOOL USE (FUNCTION CALLING)

**Note you can also do multiple tool calls at once:**

In this case, Claude will most likely try to use two separate tools, one at a time — get_weather and then get_time — in order to fully answer the user’s question. However, it will also occasionally output two tool_use blocks at once, particularly if they are not dependent on each other. You would need to execute each tool and return their results in separate tool_result blocks within a single user message.

Note that for AnthropicVertex tool calling works exacly the same

In [3]:
# Single tool call example

messages=[{"role": "user", "content": "What's the weather like in San Francisco?"}]
tools=[
        {
            "name": "get_weather",
            "description": "Get the current weather in a given location",
            "input_schema": {
                "type": "object",
                "properties": {
                    "location": {
                        "type": "string",
                        "description": "The city and state, e.g. San Francisco, CA",
                    }
                },
                "required": ["location"],
            },
        }
    ]

response = client.messages.create(
    model="claude-opus-4-20250514",
    max_tokens=1024,
    tools=tools,
    messages=messages,
)

# Append the tool call to the message

messages.append({
    "role": "assistant",
    "content": response.model_dump()['content']
})
messages

[{'role': 'user', 'content': "What's the weather like in San Francisco?"},
 {'role': 'assistant',
  'content': [{'citations': None,
    'text': "I'll check the current weather in San Francisco for you.",
    'type': 'text'},
   {'id': 'toolu_01XAqMECATPXy3iW9RcrkK81',
    'input': {'location': 'San Francisco, CA'},
    'name': 'get_weather',
    'type': 'tool_use'}]}]

In [4]:
# Let's say we execute the tool call
# Import we pass back the tool use id and the content

response_message = {
            "role": "user",
            "content": [
                {
                    "type": "tool_result",
                    "tool_use_id": "toolu_01XAqMECATPXy3iW9RcrkK81", # from the API response
                    "content": "65 degrees" # from running your tool
                }
            ]
        }

messages.append(response_message)
response = client.messages.create(
    model="claude-opus-4-20250514",
    max_tokens=1024,
    tools=tools,
    messages=messages)

response.model_dump()

{'id': 'msg_012r7NgjKAUvF7ocM2VLzNR6',
 'content': [{'citations': None,
   'text': "The current weather in San Francisco is 65 degrees Fahrenheit. It's a comfortable temperature - not too hot and not too cold!",
   'type': 'text'}],
 'model': 'claude-opus-4-20250514',
 'role': 'assistant',
 'stop_reason': 'end_turn',
 'stop_sequence': None,
 'type': 'message',
 'usage': {'cache_creation_input_tokens': 0,
  'cache_read_input_tokens': 0,
  'input_tokens': 487,
  'output_tokens': 32,
  'server_tool_use': None,
  'service_tier': 'standard'}}

### Hosting via VertexAI (Google Cloud)
Here it's important to always specify the project ID from your Google Cloud project and the region where you want to model from what region you want to run inferance on.

- Check the [Vertex AI Locations](https://cloud.google.com/vertex-ai/docs/general/locations) for more information about the specifc regions.
- The Python SDK information: https://docs.anthropic.com/en/api/claude-on-vertex-ai
- Tool calling and other functions are the same as for the base Anthropic implementation

#### NORMAL CALL

In [5]:
from anthropic import AnthropicVertex

project_id = os.getenv("GCP_PROJECT_ID")
# Where the model is running
region = os.getenv("GCP_LOCATION")
client = AnthropicVertex(project_id=project_id, region=region)

response = client.messages.create(
    model="claude-sonnet-4@20250514",
    max_tokens=100,
    messages=[
        {
            "role": "user",
            "content": "Hey Claude!",
        }
    ],
)
response.model_dump()

{'id': 'msg_vrtx_01StPvFNLWbwjDxrDjNyUrXh',
 'content': [{'citations': None,
   'text': 'Hey there! Nice to meet you. How are you doing today?',
   'type': 'text'}],
 'model': 'claude-sonnet-4-20250514',
 'role': 'assistant',
 'stop_reason': 'end_turn',
 'stop_sequence': None,
 'type': 'message',
 'usage': {'cache_creation_input_tokens': 0,
  'cache_read_input_tokens': 0,
  'input_tokens': 10,
  'output_tokens': 17,
  'server_tool_use': None,
  'service_tier': None}}

## OpenAI
### General hosting via OpenAI
Note that **it will take by default the enviroment variable OPENAI_API_KEY** as API KEY

- [Python SDK Docs](https://platform.openai.com/docs/libraries?language=python)

In [6]:
from openai import OpenAI
client = OpenAI()

response = client.responses.create(
    model="gpt-4.1",
    input="Write a one-sentence bedtime story about a unicorn."
)

response.model_dump()

{'id': 'resp_6838a579732481918e0de46b47ce20ce0daf024ae84bc2b2',
 'created_at': 1748542841.0,
 'error': None,
 'incomplete_details': None,
 'instructions': None,
 'metadata': {},
 'model': 'gpt-4.1-2025-04-14',
 'object': 'response',
 'output': [{'id': 'msg_6838a57a362081919667e093cf86f7f70daf024ae84bc2b2',
   'content': [{'annotations': [],
     'text': 'Under a moonlit sky, a gentle unicorn tiptoed across rainbow clouds, sprinkling sweet dreams over sleeping children below.',
     'type': 'output_text'}],
   'role': 'assistant',
   'status': 'completed',
   'type': 'message'}],
 'parallel_tool_calls': True,
 'temperature': 1.0,
 'tool_choice': 'auto',
 'tools': [],
 'top_p': 1.0,
 'background': False,
 'max_output_tokens': None,
 'previous_response_id': None,
 'reasoning': {'effort': None, 'generate_summary': None, 'summary': None},
 'service_tier': 'default',
 'status': 'completed',
 'text': {'format': {'type': 'text'}},
 'truncation': 'disabled',
 'usage': {'input_tokens': 18,
  'in

#### TOOL USE (FUNCTION CALLING)

In [7]:
tools = [{
    "type": "function",
    "name": "get_weather",
    "description": "Get current temperature for a given location.",
    "parameters": {
        "type": "object",
        "properties": {
            "location": {
                "type": "string",
                "description": "City and country e.g. Bogotá, Colombia"
            }
        },
        "required": [
            "location"
        ],
        "additionalProperties": False # This parameter is not available for both Anthropic and Google
    }
}]

messages = [
    {"role": "user", "content": "What is the weather like in Paris today?"}
]

response = client.responses.create(
    model="gpt-4.1",
    input=messages,
    tools=tools
)

# Note that we did not assign a role for the tool arguments provided by OpenAI
messages.extend(response.model_dump()['output'])
messages

[{'role': 'user', 'content': 'What is the weather like in Paris today?'},
 {'arguments': '{"location":"Paris, France"}',
  'call_id': 'call_wnQhXnJrWW2lXwqkOONxZoxb',
  'name': 'get_weather',
  'type': 'function_call',
  'id': 'fc_6838a57c870c8191b10b55341308b6cb0f1f7e08fa79aafe',
  'status': 'completed'}]

In [8]:
response = {
    "type": "function_call_output",
    "call_id": 'call_wnQhXnJrWW2lXwqkOONxZoxb', # from the API response
    "output": "The weather in Paris is sunny with a temperature of 20 degrees Celsius."
}

messages.append(response)
response = client.responses.create(
    model="gpt-4.1",
    input=messages,
    tools=tools
)

response.model_dump()['output']

[{'id': 'msg_6838a58218848191be69fea7d86a7a240f1f7e08fa79aafe',
  'content': [{'annotations': [],
    'text': 'The weather in Paris today is sunny with a temperature of 20°C.',
    'type': 'output_text'}],
  'role': 'assistant',
  'status': 'completed',
  'type': 'message'}]

### Hosting via Azure

Check the documentation [here](https://learn.microsoft.com/en-gb/azure/ai-services/openai/supported-languages?tabs=dotnet-secure%2Csecure%2Cpython-key%2Ccommand&pivots=programming-language-pythong)

Note that not all models are available in every region, most EU models are available in **swedencentral** and US models in **eastus2**
Also remember to deploy the model you want to use on https://ai.azure.com/

- Azure OpenAI uses the same principles for calling functions as the base OpenAI Implementation

In [9]:
from openai import AzureOpenAI

# gets the API Key from environment variable AZURE_OPENAI_API_KEY
client = AzureOpenAI(
    api_key=os.getenv("AZURE_OPENAI_API_KEY"),  
    api_version="2025-01-01-preview",
    azure_endpoint = os.getenv("AZURE_OPENAI_ENDPOINT")
    )

completion = client.chat.completions.create(
    model="gpt-4.1",  # e.g. gpt-35-instant
    messages=[
        {
            "role": "user",
            "content": "How do I output all files in a directory using Python?",
        },
    ],
)


## Google
### General Hosting (via AI Studio)

- Check the documentation on GitHub [here](https://github.com/googleapis/python-genai).

#### NORMAL CALL

In [10]:
from google import genai

client = genai.Client(api_key=os.getenv("GEMINI_API_KEY"))

response = client.models.generate_content(
    model='gemini-2.0-flash-001', contents='Why is the sky blue?'
)
response.model_dump()

{'candidates': [{'content': {'parts': [{'video_metadata': None,
      'thought': None,
      'inline_data': None,
      'code_execution_result': None,
      'executable_code': None,
      'file_data': None,
      'function_call': None,
      'function_response': None,
      'text': "The sky appears blue due to a phenomenon called **Rayleigh scattering**. Here's a breakdown of why:\n\n*   **Sunlight is composed of all colors:** White sunlight is actually a mix of all the colors of the rainbow.\n\n*   **Sunlight interacts with the atmosphere:** As sunlight enters the Earth's atmosphere, it collides with tiny air molecules (mostly nitrogen and oxygen).\n\n*   **Scattering of light:** This collision causes the sunlight to scatter in different directions. The amount of scattering depends on the wavelength (color) of the light.\n\n*   **Rayleigh scattering favors shorter wavelengths:** Rayleigh scattering is much more effective at scattering shorter wavelengths of light, like blue and violet

#### TOOL USE / FUNCTION CALLING

In [11]:
from google.genai import types

tools = [{
    "name": "get_weather",
    "description": "Get current temperature for a given location.",
    "parameters": {
        "type": "object",
        "properties": {
            "location": {
                "type": "string",
                "description": "City and country e.g. Bogotá, Colombia"
            }
        },
        "required": [
            "location"
        ],
        }
}]

messages =  [
    types.Content(
        role="user", parts=[types.Part(text="What is the weather like in Paris today?")]
    )
]

client = genai.Client(api_key=os.getenv("GEMINI_API_KEY"))
tools = types.Tool(function_declarations=tools)
config = types.GenerateContentConfig(tools=[tools])


# Send request with function declarations
response = client.models.generate_content(
     model="gemini-2.0-flash",
     contents=messages,
     config=config,
)

response.model_dump()

{'candidates': [{'content': {'parts': [{'video_metadata': None,
      'thought': None,
      'inline_data': None,
      'code_execution_result': None,
      'executable_code': None,
      'file_data': None,
      'function_call': {'id': None,
       'args': {'location': 'Paris, France'},
       'name': 'get_weather'},
      'function_response': None,
      'text': None}],
    'role': 'model'},
   'citation_metadata': None,
   'finish_message': None,
   'token_count': None,
   'finish_reason': <FinishReason.STOP: 'STOP'>,
   'url_context_metadata': None,
   'avg_logprobs': -0.002455611580184528,
   'grounding_metadata': None,
   'index': None,
   'logprobs_result': None,
   'safety_ratings': None}],
 'create_time': None,
 'response_id': None,
 'model_version': 'gemini-2.0-flash',
 'prompt_feedback': None,
 'usage_metadata': {'cache_tokens_details': None,
  'cached_content_token_count': None,
  'candidates_token_count': 7,
  'candidates_tokens_details': [{'modality': <MediaModality.TEXT:

In [12]:
messages.append(response.candidates[0].content)

# Create a function response part
function_response_part = types.Part.from_function_response(
    name="get_weather",
    response={"result": "20 degrees Celsius and sunny"},
)

messages.append(types.Content(role="user", parts=[function_response_part]))

final_response = client.models.generate_content(
    model="gemini-2.0-flash",
    contents=messages,
    config=config,
)

final_response.model_dump()['candidates'][0]

{'content': {'parts': [{'video_metadata': None,
    'thought': None,
    'inline_data': None,
    'code_execution_result': None,
    'executable_code': None,
    'file_data': None,
    'function_call': None,
    'function_response': None,
    'text': 'The weather in Paris today is 20 degrees Celsius and sunny.\n'}],
  'role': 'model'},
 'citation_metadata': None,
 'finish_message': None,
 'token_count': None,
 'finish_reason': <FinishReason.STOP: 'STOP'>,
 'url_context_metadata': None,
 'avg_logprobs': -0.0008741534625490506,
 'grounding_metadata': None,
 'index': None,
 'logprobs_result': None,
 'safety_ratings': None}

### Hosting via VertexAI
We do this using the same package as before but we now set the project id and location

In [13]:
from google import genai

# Only run this block for Vertex AI API
client = genai.Client(
    vertexai=True, project=os.environ["GCP_PROJECT_ID"], location=os.environ["GCP_LOCATION"]
)

response = client.models.generate_content(
    model='gemini-2.0-flash-001', contents='Why is the sky blue?'
)
response

GenerateContentResponse(candidates=[Candidate(content=Content(parts=[Part(video_metadata=None, thought=None, inline_data=None, code_execution_result=None, executable_code=None, file_data=None, function_call=None, function_response=None, text="The sky is blue because of a phenomenon called **Rayleigh scattering**. Here's the breakdown:\n\n*   **Sunlight and its Colors:** Sunlight is actually made up of all the colors of the rainbow.\n\n*   **Entering the Atmosphere:** When sunlight enters the Earth's atmosphere, it collides with tiny air molecules (mostly nitrogen and oxygen).\n\n*   **Scattering of Light:** This collision causes the sunlight to scatter in different directions.\n\n*   **Rayleigh Scattering:**  Rayleigh scattering is more effective at scattering shorter wavelengths of light. Blue and violet light have shorter wavelengths than other colors like red, orange, and yellow.\n\n*   **Why Blue, Not Violet?** While violet light has an even shorter wavelength than blue and is scat